In [ ]:
#NECESSARY LIBRARIES INSTALATION.
!pip install opencv-python


In [ ]:
pip install tensorflow


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import glob
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import applications
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras import backend as K
import gc
from tensorflow.keras.models import Model
import pickle

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
#DIVIDING THE DATASET INTO TRAIN TEST AND VALIDATE
import os
import shutil
import random

# Define paths
source_folder = r"C:\Users\nites\Desktop\SVPCNN"  # Root folder (A, B, ..., Z)
train_folder = r"C:\Users\nites\Desktop\svp_train"  # Already exists
test_folder = r"C:\Users\nites\Desktop\svp_test"  # Already exists
validate_folder = r"C:\Users\nites\Desktop\svp_validate"  # Already exists

# Split ratio (70% train, 15% test, 15% validate)
split_ratio = (0.7, 0.15, 0.15)

# Ensure destination subfolders (real/forged) exist
for folder in [train_folder, test_folder, validate_folder]:
    for subfolder in ["real", "forged"]:
        os.makedirs(os.path.join(folder, subfolder), exist_ok=True)

# Process each letter folder (A, B, ..., Z)
for letter in os.listdir(source_folder):
    letter_path = os.path.join(source_folder, letter)
    
    # Skip non-directories (e.g., files in SVPCNN)
    if not os.path.isdir(letter_path):
        continue
    
    print(f"Processing: {letter}")
    
    # Process real and forged subfolders
    for category in ["real", "forged"]:
        category_path = os.path.join(letter_path, category)
        
        # Skip if subfolder doesn't exist
        if not os.path.exists(category_path):
            print(f"Skipping missing folder: {category_path}")
            continue
        
        # List all files
        files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        random.shuffle(files)
        
        # Split files
        total = len(files)
        train_end = int(split_ratio[0] * total)
        test_end = train_end + int(split_ratio[1] * total)
        
        train_files = files[:train_end]
        test_files = files[train_end:test_end]
        validate_files = files[test_end:]
        
        # Move files to destination folders
        def move_files(file_list, destination):
            for file in file_list:
                src = os.path.join(category_path, file)
                dest = os.path.join(destination, category, file)
                shutil.move(src, dest)
                print(f"Moved: {file} ➔ {dest}")
        
        move_files(train_files, train_folder)
        move_files(test_files, test_folder)
        move_files(validate_files, validate_folder)

print("All files split and moved successfully!")

In [ ]:
#DELETING ALL THE UNNECCESARY FOLDERS AND ORGANIZING.
import os
import shutil

# Define the paths to your directories
svp_train_path = "C:/Users/nites/Desktop/svp_train"  # Replace with your actual path
svp_test_path = "C:/Users/nites/Desktop/svp_test"    # Replace with your actual path
svp_validate_path = "C:/Users/nites/Desktop/svp_validate"  # Replace with your actual path

# List of directories to process
directories = [svp_train_path, svp_test_path, svp_validate_path]

# Folders to keep
folders_to_keep = {'forged', 'real'}

# Function to delete unwanted folders
def clean_directory(directory):
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path) and item not in folders_to_keep:
            print(f"Deleting: {item_path}")
            shutil.rmtree(item_path)
        else:
            print(f"Keeping: {item_path}")

# Process each directory
for directory in directories:
    if os.path.exists(directory):
        print(f"Processing: {directory}")
        clean_directory(directory)
    else:
        print(f"Directory not found: {directory}")

In [ ]:
pip install --user albumentations

In [ ]:
#CONVERTING THE DATASET SPLITTING INTO STANDARD 70,15,15 SPLIT FROM THE EARLIER IMPROPER SPLIT.
import os
import shutil
import random
from sklearn.model_selection import train_test_split

# Define paths (modify as needed)
base_dir = r"C:\Users\nites\Desktop\disdataset"  # Folder containing svp_train, svp_test, svp_validate
new_dir = r"C:\Users\nites\Desktop\redisdataset"  # New folder for 70-15-15 split

# Create new directory structure
os.makedirs(os.path.join(new_dir, "train", "real"), exist_ok=True)
os.makedirs(os.path.join(new_dir, "train", "forged"), exist_ok=True)
os.makedirs(os.path.join(new_dir, "test", "real"), exist_ok=True)
os.makedirs(os.path.join(new_dir, "test", "forged"), exist_ok=True)
os.makedirs(os.path.join(new_dir, "validate", "real"), exist_ok=True)
os.makedirs(os.path.join(new_dir, "validate", "forged"), exist_ok=True)

# Collect all images from original splits
real_images = []
forged_images = []

original_splits = {
    "svp_train": "train",
    "svp_test": "test", 
    "svp_validate": "validate"
}

for orig_split, _ in original_splits.items():
    real_path = os.path.join(base_dir, orig_split, "real")
    forged_path = os.path.join(base_dir, orig_split, "forged")
    
    if os.path.exists(real_path):
        real_images.extend([os.path.join(real_path, f) for f in os.listdir(real_path)])
    if os.path.exists(forged_path):
        forged_images.extend([os.path.join(forged_path, f) for f in os.listdir(forged_path)])

# Verify we found files
if not real_images or not forged_images:
    raise FileNotFoundError("No images found in the specified paths!")

# Shuffle to ensure randomness
random.shuffle(real_images)
random.shuffle(forged_images)

# Split into 70-15-15 ratio
def split_data(images, train_ratio=0.7, test_ratio=0.15):
    train_size = int(len(images) * train_ratio)
    test_size = int(len(images) * test_ratio)
    
    train = images[:train_size]
    test = images[train_size : train_size + test_size]
    validate = images[train_size + test_size:]
    
    return train, test, validate

# Split both classes
real_train, real_test, real_validate = split_data(real_images)
forged_train, forged_test, forged_validate = split_data(forged_images)

# Copy files to new structure
def copy_files(files, split, label):
    for file in files:
        dest = os.path.join(new_dir, split, label, os.path.basename(file))
        shutil.copy2(file, dest)

print("Redistributing files...")
copy_files(real_train, "train", "real")
copy_files(forged_train, "train", "forged")
copy_files(real_test, "test", "real") 
copy_files(forged_test, "test", "forged")
copy_files(real_validate, "validate", "real")
copy_files(forged_validate, "validate", "forged")

# Print summary
print("\nRedistribution complete!")
print(f"Total real signatures: {len(real_images)}")
print(f"Total forged signatures: {len(forged_images)}")
print(f"\nNew splits:")
print(f"Train: {len(real_train) + len(forged_train)} samples")
print(f"Test: {len(real_test) + len(forged_test)} samples")
print(f"Validate: {len(real_validate) + len(forged_validate)} samples")

In [ ]:
#COUNT OF TRAIN IMAGES BEFORE PREPROCESSING.
import os

train_real_dir = r"C:\Users\nites\Desktop\redisdataset\train\real"
train_forged_dir =r"C:\Users\nites\Desktop\redisdataset\train\forged"

print(f"Number of real signatures: {len(os.listdir(train_real_dir))}")
print(f"Number of forged signatures: {len(os.listdir(train_forged_dir))}")

In [ ]:
#COUNT OF TEST IMAGES BEFORE PREPROCESSING.
import os

test_real_dir = r"C:/Users/nites/Desktop/redisdataset\test\real"
test_forged_dir = r"C:/Users/nites/Desktop/redisdataset\test\forged"

print(f"Number of real signatures: {len(os.listdir(test_real_dir))}")
print(f"Number of forged signatures: {len(os.listdir(test_forged_dir))}")

In [ ]:
#COUNT OF VALIDATE IMAGES BEFORE PREPROCESSING.
import os

val_real_dir = r"C:/Users/nites/Desktop/redisdataset\validate\real"
val_forged_dir = r"C:/Users/nites/Desktop/redisdataset\validate\forged"

print(f"Number of real signatures: {len(os.listdir(val_real_dir))}")
print(f"Number of forged signatures: {len(os.listdir(val_forged_dir))}")

In [ ]:
pip install -U albumentations

In [ ]:
#PRE-PROCESSING OF THE DATASET.

In [ ]:
import cv2
import numpy as np
import os
import albumentations as A
from skimage.filters import unsharp_mask
from tqdm import tqdm
import random
from multiprocessing import Pool, cpu_count

class EnhancedSignatureProcessor:
    def __init__(self):
        self.img_size = (224, 224)
        self.sharpen_params = {'radius': 1, 'amount': 1.5}
        
        # Augmentation pipeline
        self.train_aug = A.Compose([
            A.Affine(
                scale=(0.95, 1.05),
                rotate=(-2, 2),
                shear=(-1, 1),  
                p=0.7
            ),
            A.OneOf([
                A.GaussianBlur(blur_limit=(1, 2)),
                A.MotionBlur(blur_limit=3),
            ], p=0.4),
            A.OneOf([
                A.ISONoise(color_shift=(0.01, 0.03)),
                A.RandomGamma(gamma_limit=(90, 110)),
            ], p=0.3),
            A.ElasticTransform(alpha=0.5, sigma=10, p=0.2),
            A.RandomBrightnessContrast(
                brightness_limit=(-0.1, 0.1),
                contrast_limit=(-0.1, 0.1), 
                p=0.3
            ),
        ])

    def enhanced_preprocess(self, img):
        """Advanced preprocessing pipeline"""
        # 1. Illumination normalization
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        l = clahe.apply(l)
        lab = cv2.merge((l,a,b))
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        
        # 2. Edge-preserving denoising
        denoised = cv2.fastNlMeansDenoisingColored(img, None, h=7, 
                                                 templateWindowSize=7, 
                                                 searchWindowSize=21)
        
        # 3. Adaptive thresholding
        gray = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY_INV, 31, 2)
        
        # 4. Stroke normalization
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2))
        normalized = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        
        # 5. Context-aware sharpening
        sharpened = unsharp_mask(normalized, **self.sharpen_params)
        sharpened = (sharpened * 255).astype(np.uint8)
        
        # 6. Size normalization
        processed = self._resize_preserve_aspect(sharpened)
        return cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)

    def _resize_preserve_aspect(self, img):
        h, w = img.shape
        scale = min(self.img_size[0]/h, self.img_size[1]/w)
        new_h, new_w = int(h*scale), int(w*scale)
        resized = cv2.resize(img, (new_w, new_h))
        
        delta_w = self.img_size[1] - new_w
        delta_h = self.img_size[0] - new_h
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        return cv2.copyMakeBorder(resized, top, bottom, left, right,
                                cv2.BORDER_CONSTANT, value=255)

    def process_image(self, args):
        input_path, output_path, is_training = args
        try:
            img = cv2.imread(input_path)
            if img is None:
                return False
                
            processed = self.enhanced_preprocess(img)
            cv2.imwrite(output_path, processed)
            
            if is_training:
                base_name = os.path.splitext(output_path)[0]
                for i in range(4):
                    augmented = self.train_aug(image=processed)['image']
                    cv2.imwrite(f"{base_name}_aug{i}.jpg", augmented)
            return True
        except Exception as e:
            print(f"Error processing {input_path}: {str(e)}")
            return False

def check_class_balance(input_dir):
    class_counts = {}
    for class_name in ['real', 'forged']:
        class_dir = os.path.join(input_dir, 'train', class_name)
        count = len([f for f in os.listdir(class_dir) 
                   if f.lower().endswith(('.png','.jpg','.jpeg'))])
        class_counts[class_name] = count
    
    print("\n=== CLASS DISTRIBUTION ===")
    print(f"Real signatures: {class_counts['real']}")
    print(f"Forged signatures: {class_counts['forged']}")
    
    imbalance = abs(class_counts['real'] - class_counts['forged'])/min(class_counts.values())
    if imbalance > 0.2:
        print("\nWarning: Significant class imbalance detected!")
    return class_counts

def process_dataset(processor, input_base, output_base, num_workers=4):
    tasks = []
    
    for phase in ['train', 'test', 'val']:
        for class_name in ['real', 'forged']:
            input_dir = os.path.join(input_base, phase, class_name)
            output_dir = os.path.join(output_base, f"svp_{phase}", class_name)
            
            if not os.path.exists(input_dir):
                continue
                
            os.makedirs(output_dir, exist_ok=True)
            is_training = (phase == 'train')
            
            images = [f for f in os.listdir(input_dir) 
                     if f.lower().endswith(('.png','.jpg','.jpeg'))]
            
            for img_name in images:
                tasks.append((
                    os.path.join(input_dir, img_name),
                    os.path.join(output_dir, os.path.splitext(img_name)[0] + ".jpg"),
                    is_training
                ))
    
    with Pool(processes=min(num_workers, cpu_count())) as pool:
        results = list(tqdm(pool.imap(processor.process_image, tasks), 
                      total=len(tasks), desc="Processing Dataset"))
    
    success_rate = sum(results)/len(results)
    print(f"\nProcessing completed with {success_rate:.1%} success rate")

def verify_output_quality(output_dir):
    print("\n=== OUTPUT QUALITY CHECK ===")
    for phase in ['train', 'test', 'val']:
        for class_name in ['real', 'forged']:
            dir_path = os.path.join(output_dir, f"svp_{phase}", class_name)
            if not os.path.exists(dir_path):
                continue
                
            samples = [f for f in os.listdir(dir_path) if f.endswith('.jpg')][:3]
            print(f"\nSample outputs from {phase}/{class_name}:")
            for sample in samples:
                img = cv2.imread(os.path.join(dir_path, sample))
                print(f"  {sample}: {'Valid' if img is not None else 'Corrupt'}")

if __name__ == "__main__":
    desktop = os.path.join(os.path.expanduser("~"), "Desktop")
    input_dir = os.path.join(desktop, "redisdataset")
    output_dir = os.path.join(desktop, "svp_processed_enhanced")
    
    processor = EnhancedSignatureProcessor()
    check_class_balance(input_dir)
    process_dataset(processor, input_dir, output_dir)
    verify_output_quality(output_dir)
    
    print("\nProcessing complete! Enhanced dataset saved to:", output_dir)

In [ ]:
#COUNT OF TRAIN IMAGES AFTER PREPROCESSING AND AUGMENTING.
import os

train_real_dir = r"C:/Users/nites/Desktop/svp_processed/svp_train/real"
train_forged_dir = r"C:/Users/nites/Desktop/svp_processed/svp_train/forged"

print(f"Number of real signatures: {len(os.listdir(train_real_dir))}")
print(f"Number of forged signatures: {len(os.listdir(train_forged_dir))}")

In [ ]:
#COUNT OF TEST IMAGES AFTER PREPROCESSING.
import os

test_real_dir = r"C:/Users/nites/Desktop/svp_processed/svp_test/real"
test_forged_dir = r"C:/Users/nites/Desktop/svp_processed/svp_test/forged"

print(f"Number of real signatures: {len(os.listdir(test_real_dir))}")
print(f"Number of forged signatures: {len(os.listdir(test_forged_dir))}")

In [ ]:
#COUNT OF VALIDATE IMAGES AFTER PREPROCESSING.
import os

val_real_dir = r"C:/Users/nites/Desktop/svp_processed/svp_validate/real"
val_forged_dir = r"C:/Users/nites/Desktop/svp_processed/svp_validate/forged"

print(f"Number of real signatures: {len(os.listdir(val_real_dir))}")
print(f"Number of forged signatures: {len(os.listdir(val_forged_dir))}")

In [ ]:
#DISPLAYING THE IMAGES AFTER PREPROCESSING THROUGH VISUALIZATION OF PREPROCESSED AND AUGMENTED DATA OF ANY RANDOMLY CHOOSEN IMAGE IN THE DATASET...
import cv2
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from skimage.filters import unsharp_mask

def process_and_visualize(image_path):
    """Enhanced visualization of preprocessing and augmentations"""
    # 1. Load original image
    original = cv2.imread(image_path)
    if original is None:
        print(f"Error: Could not load image from {image_path}")
        return
    
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

    # 2. Enhanced preprocessing function
    def preprocess(img):
        # Noise reduction
        denoised = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
        
        # Adaptive thresholding
        gray = cv2.cvtColor(denoised, cv2.COLOR_RGB2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                      cv2.THRESH_BINARY_INV, 31, 7)
        
        # Sharpening
        sharpened = unsharp_mask(thresh, radius=1.5, amount=2.0)
        sharpened = (sharpened * 255).astype(np.uint8)
        
        # Morphological cleanup
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
        cleaned = cv2.morphologyEx(sharpened, cv2.MORPH_CLOSE, kernel)
        
        # Resize and convert to RGB
        resized = cv2.resize(cleaned, (224, 224))
        return cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)

    # 3. Apply preprocessing
    preprocessed = preprocess(original)

    # 4. Define complete augmentation pipeline
    augmentations = A.Compose([
        A.Rotate(limit=3, p=0.5),
        A.ElasticTransform(alpha=1.2, sigma=20, p=0.4),
        A.OpticalDistortion(distort_limit=0.2, shift_limit=0.1, p=0.3),
        A.GaussianBlur(blur_limit=(1, 3), p=0.3),
        A.RandomBrightnessContrast(
            brightness_limit=(-0.15, 0.15),
            contrast_limit=(-0.15, 0.15), p=0.4),
        A.GridDistortion(p=0.2),
        A.CoarseDropout(max_holes=8, max_height=10, max_width=10, p=0.3),
    ])

    # 5. Generate 6 augmented versions
    augmented_images = []
    for _ in range(6):
        augmented = augmentations(image=preprocessed)['image']
        augmented_images.append(augmented)

    # 6. Display results in grid layout
    plt.figure(figsize=(20, 10))
    
    # Original
    plt.subplot(2, 4, 1)
    plt.imshow(original)
    plt.title("Original\n(Original Input)")
    plt.axis('off')
    
    # Preprocessed
    plt.subplot(2, 4, 2)
    plt.imshow(preprocessed)
    plt.title("Preprocessed\n(After Enhancement)")
    plt.axis('off')
    
    # Augmentations
    for i, img in enumerate(augmented_images, 3):
        plt.subplot(2, 4, i)
        plt.imshow(img)
        plt.title(f"Augmented #{i-2}\n(Random Combination)")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Example usage:
if __name__ == "__main__":
    image_path = r"C:\Users\nites\Desktop\redisdataset\train\real\original_21_1.png"  
    process_and_visualize(image_path)

In [ ]:
# 1. AUC-ROC (Area Under the Curve - Receiver Operating Characteristic)
# Ideal Value: Higher is better (Close to 1.0).

# 2. Loss (Binary Cross-Entropy)
# Ideal Value: Lower is better (Close to 0).

# 3. Precision
# Ideal Value: Higher is better (Close to 1.0).

# 4. Recall (Sensitivity)
# Ideal Value: Higher is better (Close to 1.0).


In [ ]:
pip install tf-keras-vis

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers, regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import GlobalAveragePooling2D

# Set all seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'
tf.config.experimental.enable_op_determinism()

def create_data_generators():
    """Create enhanced data generators with validation split"""
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.2,
        brightness_range=[0.9, 1.1],
        horizontal_flip=True,
        fill_mode='reflect',
        validation_split=0.2
    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    return train_datagen, test_datagen

def load_data_with_validation(data_dir="C:/Users/nites/Desktop/svp_processed"):
    """Load data with proper validation split"""
    data_dir = os.path.expanduser(data_dir)
    
    train_datagen, test_datagen = create_data_generators()
    
    # Training generator (80% of training data)
    train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'svp_train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        color_mode='rgb',
        subset='training',
        shuffle=True,
        seed=42
    )
    
    # Validation generator (20% of training data)
    val_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'svp_train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        color_mode='rgb',
        subset='validation',
        shuffle=True,
        seed=42
    )
    
    # Test generator
    test_generator = test_datagen.flow_from_directory(
        os.path.join(data_dir, 'svp_test'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        color_mode='rgb',
        shuffle=False
    )
    
    # Verify data loading
    print("\n=== DATA VERIFICATION ===")
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")
    print(f"Test samples: {test_generator.samples}")
    
    # Check class distribution
    print("\nClass indices:", train_generator.class_indices)
    print("Training class counts:", np.bincount(train_generator.classes))
    print("Validation class counts:", np.bincount(val_generator.classes))
    print("Test class counts:", np.bincount(test_generator.classes))
    
    return train_generator, val_generator, test_generator

def build_optimized_model(input_shape=(224, 224, 3)):
    """Build optimized hybrid model with fine-tuning capability"""
    # Load pre-trained EfficientNet
    base_model = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    
    # Freeze initial layers
    for layer in base_model.layers[:100]:
        layer.trainable = False
        
    # Unfreeze later layers
    for layer in base_model.layers[100:]:
        layer.trainable = True
    
    # Create new model with more sophisticated architecture
    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu', 
                    kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    
    # Custom optimizer configuration with warmup
    optimizer = optimizers.Adam(
        learning_rate=0.0001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True
    )
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.AUC(name='prc', curve='PR'),
            tf.keras.metrics.TruePositives(name='tp'),
            tf.keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.FalseNegatives(name='fn')
        ]
    )
    
    return model

def get_enhanced_callbacks():
    """Return optimized callbacks with model checkpointing"""
    return [
        callbacks.EarlyStopping(
            monitor='val_auc',
            patience=15,
            mode='max',
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        callbacks.ModelCheckpoint(
            'best_model.keras',
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            save_weights_only=False,
            verbose=1
        ),
        callbacks.TensorBoard(
            log_dir='./logs',
            histogram_freq=1,
            update_freq='epoch'
        ),
        callbacks.CSVLogger('training_log.csv')
    ]

def plot_training_history(history):
    """Plot training metrics with enhanced visualization"""
    plt.figure(figsize=(20, 8))
    
    # Plot accuracy
    plt.subplot(2, 3, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs', pad=10)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(2, 3, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs', pad=10)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot AUC
    plt.subplot(2, 3, 3)
    plt.plot(history.history['auc'], label='Train AUC')
    plt.plot(history.history['val_auc'], label='Validation AUC')
    plt.title('ROC AUC Over Epochs', pad=10)
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()
    
    # Plot Precision-Recall AUC
    plt.subplot(2, 3, 4)
    plt.plot(history.history['prc'], label='Train PR AUC')
    plt.plot(history.history['val_prc'], label='Validation PR AUC')
    plt.title('PR AUC Over Epochs', pad=10)
    plt.xlabel('Epoch')
    plt.ylabel('PR AUC')
    plt.legend()
    
    # Plot Precision
    plt.subplot(2, 3, 5)
    plt.plot(history.history['precision'], label='Train Precision')
    plt.plot(history.history['val_precision'], label='Validation Precision')
    plt.title('Precision Over Epochs', pad=10)
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend()
    
    # Plot Recall
    plt.subplot(2, 3, 6)
    plt.plot(history.history['recall'], label='Train Recall')
    plt.plot(history.history['val_recall'], label='Validation Recall')
    plt.title('Recall Over Epochs', pad=10)
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def evaluate_model(model, test_generator):
    """Comprehensive model evaluation with enhanced metrics"""
    print("\n=== FINAL EVALUATION ===")
    
    # Load best weights
    if os.path.exists('best_model.keras'):
        model = models.load_model('best_model.keras')
        print("Loaded best saved model for evaluation")
    
    # Evaluate on test set
    results = model.evaluate(test_generator, verbose=0)
    metrics = {
        'Test Loss': results[0],
        'Test Accuracy': results[1],
        'Test Precision': results[2],
        'Test Recall': results[3],
        'Test ROC AUC': results[4],
        'Test PR AUC': results[5]
    }
    
    # Calculate additional metrics
    tp = results[6]
    fp = results[7]
    tn = results[8]
    fn = results[9]
    
    # Add derived metrics
    metrics['Test F1 Score'] = 2 * (metrics['Test Precision'] * metrics['Test Recall']) / \
                              (metrics['Test Precision'] + metrics['Test Recall'] + 1e-7)
    metrics['Test Specificity'] = tn / (tn + fp + 1e-7)
    
    for name, value in metrics.items():
        print(f"{name}: {value:.4f}")
    
    # Generate predictions
    y_pred = (model.predict(test_generator) > 0.5).astype(int)
    y_true = test_generator.classes
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, 
                              target_names=['Forged', 'Genuine'],
                              zero_division=0))
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Forged', 'Genuine'],
                yticklabels=['Forged', 'Genuine'],
                annot_kws={"size": 16})
    plt.title('Confusion Matrix', fontsize=14)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.show()
    
    # ROC and PR curves
    y_scores = model.predict(test_generator)
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    # PR Curve
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    pr_auc = auc(recall, precision)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC Curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('Receiver Operating Characteristic', fontsize=14)
    plt.legend(loc="lower right")
    
    plt.subplot(1, 2, 2)
    plt.plot(recall, precision, color='blue', lw=2,
             label=f'PR Curve (AUC = {pr_auc:.2f})')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.title('Precision-Recall Curve', fontsize=14)
    plt.legend(loc="upper right")
    
    plt.tight_layout()
    plt.show()

def train_and_evaluate():
    # Load data
    train_gen, val_gen, test_gen = load_data_with_validation()
    
    # Calculate class weights
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_gen.classes),
        y=train_gen.classes
    )
    class_weights = {i: w for i, w in enumerate(class_weights)}
    print("\nClass weights:", class_weights)
    
    # Build model
    model = build_optimized_model()
    
    print("\n=== MODEL SUMMARY ===")
    model.summary()
    
    # Train model
    print("\n=== TRAINING STARTED ===")
    history = model.fit(
        train_gen,
        steps_per_epoch=len(train_gen),
        epochs=50,  # Increased epochs for better convergence
        validation_data=val_gen,
        validation_steps=len(val_gen),
        callbacks=get_enhanced_callbacks(),
        class_weight=class_weights,
        verbose=1
    )
    
    # Plot training history
    plot_training_history(history)
    
    # Evaluate model
    evaluate_model(model, test_gen)

if __name__ == "__main__":
    train_and_evaluate()

In [ ]:
#RANDOMLY CHOOSING ANY IMAGE FROM PREPROCESSED FOLDER AND DISPLAYING THE RESULT ALONG THE CHOOSEN IMAGE WITH ITS ACTUALL LABELLING ..
import os
import random
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt  # Replaces OpenCV for image display

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# Load model with compile=False to avoid warnings
model = load_model('best_model.h5', compile=False)
# Recompile with minimal metrics if needed
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

def get_random_image(dataset_path):
    classes = ['real', 'forged']
    chosen_class = random.choice(classes)
    class_path = os.path.join(dataset_path, chosen_class)
    
    images = [f for f in os.listdir(class_path) 
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    return os.path.join(class_path, random.choice(images)), chosen_class

def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(
        image_path, target_size=(224, 224))  # RGB auto-conversion
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return np.expand_dims(img, axis=0)  # Add batch dimension

def predict_and_display():
    image_path, true_label = get_random_image("C:/Users/nites/Desktop/svp_processed/svp_train")
    img_array = preprocess_image(image_path)
    
    prediction = model.predict(img_array, verbose=0)[0][0]
    predicted_label = "real" if prediction > 0.3 else "forged"
    
    print(f"Image: {os.path.basename(image_path)}")
    print(f"True: {true_label} | Predicted: {predicted_label} ({prediction:.4f})")
    print("✓ Correct" if predicted_label == true_label else "✗ Incorrect")
    
    # Display with matplotlib (no OpenCV dependency)
    plt.imshow(plt.imread(image_path))
    plt.title(f"True: {true_label} | Predicted: {predicted_label}")
    plt.axis('off')
    plt.show()

predict_and_display()

In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load model
model = load_model("best_model.keras")

def predict(image):
    # Convert and preprocess image
    img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (224, 224)) / 255.0
    
    # Get prediction
    pred = model.predict(np.expand_dims(img, axis=0), verbose=0)[0][0]
    confidence = max(pred, 1 - pred) * 100  # Convert to percentage
    
    # Determine result
    if pred > 0.7:
        return "REAL SIGNATURE", f"{confidence:.1f}%", "green"
    else:
        return "FORGED SIGNATURE", f"{confidence:.1f}%", "red"

# Custom CSS
css = """
.green-box {
    background: #e6f7e6 !important; 
    border: 2px solid #4CAF50 !important;
    padding: 20px !important;
    border-radius: 5px !important;
}
.red-box {
    background: #ffebee !important;
    border: 2px solid #F44336 !important;
    padding: 20px !important;
    border-radius: 5px !important;
}
.result-text {
    font-size: 20px !important;
    font-weight: bold !important;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🔍 Signature Verification System")
    
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="Upload Signature", type="numpy")
            submit_btn = gr.Button("Verify", variant="primary")
        
        with gr.Column():
            with gr.Group():
                result_box = gr.Textbox(label="Result", interactive=False, elem_classes="result-text")
                confidence_box = gr.Textbox(label="Confidence", interactive=False)
    
    def update_ui(image):
        label, confidence, color = predict(image)
        return (
            gr.Textbox(value=label, elem_classes=color+"-box"), 
            gr.Textbox(value=confidence)
        )
    
    submit_btn.click(
        fn=update_ui,
        inputs=image_input,
        outputs=[result_box, confidence_box]
    )

demo.launch(share=True)
   

In [ ]:
pip freeze > requirements.txt
